In [1]:
from simulator import blogm, bSqc, Neg, Sa
from mpo import tel_mpo
import torch
from math import prod
from functools import reduce
import pandas
from utils import dtype, device, pauli, basis, torch_data

In [4]:
seed = 0
train, test = True, True
file = f'seed{seed}'
train_ratio = 5/6
batch = 500

mdl = tel_mpo(16, bond=10)
total=0 # find size of the model
for p in mdl.parameters():
    total+=prod(p.shape)
total#, True_fid(mdl, psi)

3200

In [5]:
for N in range(4, 18, 2):
    torch.manual_seed(seed)
    prepseq, shadow_state, rhoS = torch_data(f'../data/post_selected/data_{N}pa.pickle', shuffle=True)
    
    prepseq, shadow_state, rhoS = prepseq[:600000], shadow_state[:600000], rhoS[:600000]
    
    train_size = int(prepseq.shape[0]*train_ratio)
    test_size = prepseq.shape[0]-train_size
    
    prepseq_train, prepseq_test = prepseq[:train_size], prepseq[train_size:]
    shadow_state_train, shadow_state_test = shadow_state[:train_size], shadow_state[train_size:]
    rhoS_train, rhoS_test = rhoS[:train_size], rhoS[train_size:]
    
    # split in batches
    prepseq_train = prepseq_train.view(-1, batch, N-2)
    shadow_state_train = shadow_state_train.view(-1, batch, 4)
    rhoS_train = rhoS_train.view(-1, batch, 4, 4)

    prepseq_test = prepseq_test.view(-1, batch, N-2)
    shadow_state_test = shadow_state_test.view(-1, batch, 4)
    rhoS_test = rhoS_test.view(-1, batch, 4, 4)
    
    # load new model
    if N == 4:
        mdl = tel_mpo(34, bond=10)
    else:
        mdl.load_state_dict(torch.load(f'{file}/models/mpo_N={N-2}_pa.pt'))
    
    # load old model
    # mdl.load_state_dict(torch.load(f'{file}/models/mpo_N={N}_na.pt'))
    
    optimizer = torch.optim.Adam(mdl.parameters(), lr=1e-3) # 0.0001
    l = {'train Sqc':[], 'test Sqc':[], 'test Neg':[], 'test Sa':[], 'loss':[]}
    
    for epoch in range(10):
        # Train
        if train:
            print('='*50+'   Train   '+'='*50)
            mdl.train()
            for i in range(prepseq_train.shape[0]):
                rhoC = mdl(prepseq_train[i])
                l['train Sqc'].append(bSqc(rhoS_train[i], rhoC).mean().item())
                optimizer.zero_grad()
                probs = torch.bmm(torch.bmm(shadow_state_train[i].unsqueeze(1), rhoC), shadow_state_train[i].conj().unsqueeze(-1)).view(-1).real
                loss = -probs.log().mean()
                loss.backward()
                optimizer.step()
                l['loss'].append(loss.item())
                if (i+1)%100 == 0:
                    trainS = torch.tensor(l['train Sqc'])[-i:].mean().item()
                    loss = torch.tensor(l['loss'])[-i:].mean().item()
                    print('epoch:  %3d | step:  %3d | N:  %d | train Sqc: %.4f | loss: %.4f' %(epoch, i, N, trainS, loss))
        # Test
        if test:
            with torch.no_grad():
                print('='*50+'   Test   '+'='*50)
                mdl.eval()
                for i in range(prepseq_test.shape[0]):
                    rhoC = mdl(prepseq_test[i])
                    l['test Sqc'].append(bSqc(rhoS_test[i], rhoC).mean().item())
                    l['test Neg'].append(Neg(rhoS_test[i], rhoC).mean().item())
                    l['test Sa'].append(Sa(rhoS_test[i], rhoC).mean().item())
                    if (i+1)%100 == 0:
                        testS = torch.tensor(l['test Sqc'])[-i:].mean().item()
                        testN = torch.tensor(l['test Neg'])[-i:].mean().item()
                        print('epoch:  %3d | step:  %3d | N:  %d | test Sqc: %.4f | test Neg: %.4f' %(epoch, i, N, testS, testN))
        torch.save(l, f'{file}/record/mpo_N={N}_pa.pt')
        torch.save(mdl.state_dict(), f'{file}/models/mpo_N={N}_pa.pt')

==================================================   Train   ==================================================
epoch:    0 | step:   99 | N:  4 | train Sqc: 1.3189 | loss: 1.3808
epoch:    0 | step:  199 | N:  4 | train Sqc: 1.2028 | loss: 1.3647
epoch:    0 | step:  299 | N:  4 | train Sqc: 1.0759 | loss: 1.3461
epoch:    0 | step:  399 | N:  4 | train Sqc: 0.9574 | loss: 1.3272
epoch:    0 | step:  499 | N:  4 | train Sqc: 0.8580 | loss: 1.3108
epoch:    0 | step:  599 | N:  4 | train Sqc: 0.7858 | loss: 1.2984
epoch:    0 | step:  699 | N:  4 | train Sqc: 0.7313 | loss: 1.2891
epoch:    0 | step:  799 | N:  4 | train Sqc: 0.6887 | loss: 1.2818
epoch:    0 | step:  899 | N:  4 | train Sqc: 0.6551 | loss: 1.2761
epoch:    0 | step:  999 | N:  4 | train Sqc: 0.6289 | loss: 1.2716
==================================================   Test   ==================================================
epoch:    0 | step:   99 | N:  4 | test Sqc: 0.3713 | test Neg: 0.4154
epoch:    0 | step:  199 |

FileNotFoundError: [Errno 2] No such file or directory: '../data/post_selected/data_18pa.pickle'